## Project Assignment 
### Machile Learning
Eliana Ferreira


In [26]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report 
import matplotlib.pyplot as plt
import seaborn as sns

### Data Import
Importar o dataset dado

In [ ]:
df = pd.read_csv('RTA Dataset.csv')
print(df)
#df_original = df.copy()

In [ ]:
df_cleaned = pd.read_csv('cleaned.csv')
print(df_cleaned)

Calcular Variância Explicada de cada Feature 

### Exploratory Data Analysis


In [ ]:
df.shape

In [ ]:
slight_injuries = (df['Accident_severity'] == 'Slight Injury').sum()
serious_injuries = (df['Accident_severity'] == 'Serious Injury').sum()
fatal_injuries = (df['Accident_severity'] == 'Fatal injury').sum()
num_linhas = len(df)
print(slight_injuries)
print(serious_injuries)
print(fatal_injuries)
print(num_linhas)


In [ ]:
#saber as classes de cada feature e o tamanho de cada classe
labels = df['Type_of_vehicle'].unique()
print(labels)
sizes = []

for label in labels:
    sizes.append((df['Type_of_vehicle'] == label).sum())
print(sizes)


In [ ]:
#visualizar pie chart
labels = ['Slight Injuries', 'Serious Injuries', 'Fatal Injuries']
sizes = [slight_injuries, serious_injuries, fatal_injuries]
colors = ['#66c2a5', '#fc8d62', '#8da0cb']  
explode = (0.1, 0.1, 0.1)  

plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, autopct='%1.0f%%', colors=colors, explode=explode)
plt.title('Severidade da Lesão')
plt.axis('equal')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

categorical_cols = df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    if len(df[col].unique()) < 10:
        plt.figure(figsize=(6, 4))
        sns.countplot(data=df, x=col)
        plt.title(f'Distribution of {col}')
        plt.xticks(rotation=45)
        plt.show()

for col in categorical_cols:
    if len(df[col].unique()) < 10:
        plt.figure(figsize=(6, 6))
        df[col].value_counts().plot.pie(autopct='%1.1f%%', startangle=90)
        plt.title(f'{col} Distribution')
        plt.ylabel('')
        plt.show()


## Data pre-processing 

### Missing Values

In [ ]:
#imprimir missing values
print (df.isnull().sum())
totalnull = df.isnull().sum().sum()
print(totalnull)

In [35]:
#função para missing values
def tratar_missing_values(df, threshold=0.1): 
    colunas = []
    for col in df.columns:
       
        missing_percentage = df[col].isnull().mean() #aplica-se a média porque é um booleano, por isso faz true/todos
        # se a percentagem de missing values for maior que 10% faz-se drop à coluna
        if missing_percentage > threshold:
            colunas.append(col)

        else:
            if 'Unknown'in df[col].unique(): 
                df[col].fillna('Unknown', inplace=True) #o inplace=True evita a criação de uma nova variável, substituindo diretamente 
            else:  
                mode_value = df[col].mode()[0]
                df[col].fillna(mode_value, inplace=True)
            
    df = df.drop(columns=colunas)
    
    return df, colunas






### Values = 'na'

In [ ]:
#comprender 'na'
na_count_df = (df == 'na').sum()
print (na_count_df)

for column in df.columns:
    na_indices = df[df[column] == 'na'].index
    if len(na_indices) > 0:
        print(f"Índices na coluna '{column}' com valor 'na': {na_indices.tolist()}")


#em 4 colunas, mesmo número, mesmos índices - relacionados com as casualties

### Turn categorical variables into numerical variables

#### Scenario A (Binary Classification)

In [37]:
# tornar as variáveis categóricas em variáveis numéricas


def tornarnuméricoA(df, colunas):
    # Converter a hora o nº dos minutos desde a meia-noite
    if 'Time' not in colunas:
        df['Time'] = df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.hour * 60 + \
                                            pd.to_datetime(df['Time'], format='%H:%M:%S').dt.minute
    
    if 'Day_of_week' not in colunas:
        df['Day_of_week'] = df['Day_of_week'].replace({"Monday" : 0,
                                                               "Tuesday": 1,
                                                               "Wednesday": 2,
                                                               "Thursday": 3,
                                                               "Friday": 4,
                                                               "Saturday": 5,
                                                               "Sunday": 6})
    if 'Age_band_of_driver' not in colunas:
        df['Age_band_of_driver'] = df['Age_band_of_driver'].replace({"Unknown" : 0,
                                                                             "Under 18": 1,
                                                                             "18-30": 2,
                                                                             "31-50": 3,
                                                                             "Over 51": 4})
    if 'Sex_of_driver' not in colunas:
        df['Sex_of_driver'] = df['Sex_of_driver'].replace({"Unknown": 0,
                                                                   "Female": 1,
                                                                   "Male": 2})
    if 'Educational_level' not in colunas:
        df['Educational_level'] = df['Educational_level'].replace({"Unknown": 0,
                                                                           "Illiterate": 1,
                                                                           "Writing & reading": 2,
                                                                           "Elementary school": 3,
                                                                           "Junior high school": 4,
                                                                           "High school": 5,
                                                                           "Above high school": 6})
    if 'Vehicle_driver_relation' not in colunas:
        df['Vehicle_driver_relation'] = df['Vehicle_driver_relation'].replace({"Unknown": 0,
                                                                                       "Other": 0,
                                                                                       "Employee": 1,
                                                                                       "Owner": 2})
    
    if 'Driving_experience' not in colunas:

        df['Driving_experience'] = df['Driving_experience'].replace({"unknown": 0,
                                                                             "No Licence": 1,
                                                                             "Below 1yr": 2,
                                                                             "1-2yr": 3,
                                                                             "2-5yr": 4,
                                                                             "5-10yr": 5,
                                                                             "Above 10yr": 6})
    if 'Type_of_vehicle' not in colunas:
        df['Type_of_vehicle'] = df['Type_of_vehicle'].replace({"Other": 0,
                                                                       "Public (> 45 seats)": 1,
                                                                       "Public (13?45 seats)": 1,
                                                                       "Public (12 seats)": 1,
                                                                       "Lorry (41?100Q)" : 2,
                                                                       "Lorry (11?40Q)": 2,
                                                                       "Long lorry": 2,
                                                                       "Automobile": 3,
                                                                       "Taxi": 4,
                                                                       "Pick up upto 10Q": 5,
                                                                       "Stationwagen": 6,
                                                                       "Ridden horse": 7,
                                                                       "Bajaj": 8,
                                                                       "Motorcycle": 8,
                                                                       "Turbo":9,
                                                                       "Special vehicle" : 10,
                                                                       "Bicycle": 11})
    if 'Owner_of_vehicle' not in colunas:
        df['Owner_of_vehicle'] = df['Owner_of_vehicle'].replace({"Other": 0,
                                                                         "Owner": 1,
                                                                         "Organization": 2,
                                                                         "Governmental": 3})
    
    if 'Area_accident_occured' not in colunas:
        df['Area_accident_occured'] = df['Area_accident_occured'].replace({"Unknown": 0,
                                                                                   "Other": 0,
                                                                                   "Rural village areasOffice areas": 0,
                                                                                   "  Recreational areas": 1,
                                                                                   "Recreational areas": 1,
                                                                                   "Residential areas": 2,
                                                                                   "Office areas": 3,
                                                                                   " Industrial areas": 4,
                                                                                   " Church areas": 5,
                                                                                   "  Market areas": 6,
                                                                                   "Rural village areas": 7,
                                                                                   " Outside rural areas": 8,
                                                                                   " Hospital areas": 9,
                                                                                  "School areas": 10})
    if 'Service_year_of_vehicle' not in colunas:    
        df['Service_year_of_vehicle'] = df['Service_year_of_vehicle'].replace({"Unknown" : 0,
                                                                           "Below 1yr" : 1,
                                                                           "1-2yr" : 2,
                                                                           "2-5yrs" : 3,
                                                                           "5-10yrs" : 4,
                                                                           "Above 10yr" : 5})
    if 'Defect_of_vehicle' not in colunas:    
        df['Defect_of_vehicle'] = df['Defect_of_vehicle'].replace({"No defect" : 0,
                                                               "5" : 1,
                                                               "7" : 2})
        
    if 'Lanes_or_Medians' not in colunas: 
        df['Lanes_or_Medians'] = df['Lanes_or_Medians'].replace({"Unknown" : 0,
                                                                         "other": 0,
                                                                         "Undivided Two way": 1,
                                                                         "Double carriageway (median)": 2,
                                                                         "Two-way (divided with solid lines road marking)": 3,
                                                                         "Two-way (divided with broken lines road marking)": 4,
                                                                         "One way": 5})
    if 'Road_allignment' not in colunas: 
        df['Road_allignment'] = df['Road_allignment'].replace({"Tangent road with flat terrain": 0,
                                                                       "Tangent road with mild grade and flat terrain": 1,
                                                                       "Tangent road with rolling terrain": 2,
                                                                       "Tangent road with mountainous terrain and": 3,
                                                                       "Escarpments": 4,
                                                                       "Gentle horizontal curve": 5,
                                                                       "Steep grade downward with mountainous terrain": 6,
                                                                       "Sharp reverse curve": 7,
                                                                       "Steep grade upward with mountainous terrain": 8})
    
    if 'Types_of_Junction' not in colunas: 
        df['Types_of_Junction'] = df['Types_of_Junction'].replace({"Unknown": 0,
                                                                           "Other": 0,
                                                                           "No junction": 1,
                                                                           "Y Shape": 2,
                                                                           "Crossing": 3,
                                                                           "O Shape": 4,
                                                                           "T Shape": 5,
                                                                           "X Shape": 6})
    if 'Road_surface_type' not in colunas:
        df['Road_surface_type'] = df['Road_surface_type'].replace({"Other": 0,
                                                                           "Asphalt roads": 1, 
                                                                           "Asphalt roads with some distress": 2,
                                                                           "Gravel roads": 3,
                                                                           "Earth roads": 4})
    
    if 'Road_surface_conditions' not in colunas:
        df['Road_surface_conditions'] = df['Road_surface_conditions'].replace({"Dry": 0,
                                                                                       "Wet or damp": 1,
                                                                                       "Snow": 2,
                                                                                       "Flood over 3cm. deep": 3})
    
    if 'Light_conditions' not in colunas:
        df['Light_conditions'] = df['Light_conditions'].replace({"Daylight": 0,
                                                                         "Darkness - lights lit": 1,
                                                                         "Darkness - no lighting": 2,
                                                                         "Darkness - lights unlit": 3})
    if 'Weather_conditions' not in colunas:
        df['Weather_conditions'] = df['Weather_conditions'].replace({"Unknown": 0,
                                                                             "Other": 0,
                                                                             "Normal": 1,
                                                                             "Raining": 2,
                                                                             "Raining and Windy": 3,
                                                                             "Cloudy": 4,
                                                                             "Windy": 5,
                                                                             "Snow": 6,
                                                                             "Fog or mist": 7})
    if 'Type_of_collision' not in colunas:
        df['Type_of_collision'] = df['Type_of_collision'].replace({"Unknown": 0,
                                                                           "Other": 0,
                                                                           "Collision with roadside-parked vehicles":1,
                                                                           "Vehicle with vehicle collision":2,
                                                                           "Collision with roadside objects":3,
                                                                           "Collision with animals": 4,
                                                                           "Rollover": 5,
                                                                           "Fall from vehicles": 6,
                                                                           "Collision with pedestrians": 7,
                                                                           "With Train": 8})
    if 'Vehicle_movement' not in colunas:
        df['Vehicle_movement'] = df['Vehicle_movement'].replace({"Unknown": 0,
                                                                         "Other": 0,
                                                                         "Going straight": 1,
                                                                         "U-Turn": 2,
                                                                         "Moving Backward": 3,
                                                                         "Turnover": 4,
                                                                         "Waiting to go": 5,
                                                                         "Getting off": 6,
                                                                         "Reversing": 7,
                                                                         "Parked": 8,
                                                                         "Stopping": 9,
                                                                         "Overtaking": 10,
                                                                         "Entering a junction": 11})
    
    if 'Casualty_class' not in colunas:
        df['Casualty_class'] = df['Casualty_class'].replace({"na": 0,
                                                                     "Driver or rider": 1,
                                                                     "Pedestrian": 2,
                                                                     "Passenger": 3})
    
    if 'Sex_of_casualty' not in colunas:
        df['Sex_of_casualty'] = df['Sex_of_casualty'].replace({"na": 0,
                                                                       "Female": 1,
                                                                       "Male":2})
    if 'Age_band_of_casualty' not in colunas:
        df['Age_band_of_casualty'] = df['Age_band_of_casualty'].replace({"na":0,
                                                                                 "5": 1,
                                                                                 "Under 18": 1,
                                                                                 "18-30": 2,
                                                                                 "31-50": 3,
                                                                                 "Over 51": 4})
    
    if 'Casualty_severity' not in colunas:
        df['Casualty_severity'] = df['Casualty_severity'].replace({"na": 0,
                                                                           "1": 1,
                                                                           "2": 2,
                                                                           "3": 3})
    if 'Work_of_casuality' not in colunas:
        df['Work_of_casuality'] = df['Work_of_casuality'].replace({"Unknown" : 0,
                                                               "Other": 0,
                                                               "Student": 1,
                                                               "Unemployed": 2,
                                                               "Employee": 3,
                                                               "Driver": 4,
                                                               "Self-employed": 5})
        
    if 'Fitness_of_casuality' not in colunas:
        df['Fitness_of_casuality'] = df['Fitness_of_casuality'].replace({"Other" : 0,
                                                                     "Normal" : 1,
                                                                     "NormalNormal": 1,
                                                                     "Deaf" : 3,
                                                                     "Blind" : 4})
    
    if 'Pedestrian_movement' not in colunas:
        df['Pedestrian_movement'] = df['Pedestrian_movement'].replace({"Unknown or other":0,
                                                                    "Not a Pedestrian": 1,
                                                                    "Crossing from driver's nearside": 2,
                                                                    "Crossing from nearside - masked by parked or statioNot a Pedestrianry vehicle": 3,
                                                                    "Crossing from offside - masked by  parked or statioNot a Pedestrianry vehicle": 4,
                                                                    "In carriageway, statioNot a Pedestrianry - not crossing  (standing or playing)": 5,
                                                                    "Walking along in carriageway, back to traffic": 6,
                                                                    "Walking along in carriageway, facing traffic": 7,
                                                                    "In carriageway, statioNot a Pedestrianry - not crossing  (standing or playing) - masked by parked or statioNot a Pedestrianry vehicle": 8})
    
    if 'Cause_of_accident' not in colunas:
        df['Cause_of_accident'] = df['Cause_of_accident'].replace({"Unknown": 0,
                                                                           "Other": 0,
                                                                           "Moving Backward": 1,
                                                                           "Overtaking": 2,
                                                                           "Changing lane to the left": 3,
                                                                           "Changing lane to the right": 4,
                                                                           "Overloading": 5,
                                                                           "No priority to vehicle": 6,
                                                                           "No priority to pedestrian": 7,
                                                                           "No distancing": 8,
                                                                           "Getting off the vehicle improperly": 9,
                                                                           "Improper parking": 10,
                                                                           "Overspeed": 11,
                                                                           "Driving carelessly": 12,
                                                                           "Driving at high speed": 13,
                                                                           "Driving to the left": 14,
                                                                           "Overturning": 15,
                                                                           "Turnover": 16,
                                                                           "Driving under the influence of drugs": 17,
                                                                           "Drunk driving": 18})
    if 'Accident_severity' not in colunas:
        df['Accident_severity'] = df['Accident_severity'].replace({"Slight Injury": 0,
                                                                           "Serious Injury": 1,
                                                                           "Fatal injury": 1})


    print(df)
    return df




Scenario B (Multiclass Classification)

In [38]:
# tornar as variáveis categóricas em variáveis numéricas


def tornarnuméricoB(df, colunas):
    # Converter a hora o nº dos minutos desde a meia-noite
    if 'Time' not in colunas:
        df['Time'] = df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.hour * 60 + \
                                            pd.to_datetime(df['Time'], format='%H:%M:%S').dt.minute
    
    if 'Day_of_week' not in colunas:
        df['Day_of_week'] = df['Day_of_week'].replace({"Monday" : 0,
                                                               "Tuesday": 1,
                                                               "Wednesday": 2,
                                                               "Thursday": 3,
                                                               "Friday": 4,
                                                               "Saturday": 5,
                                                               "Sunday": 6})
    if 'Age_band_of_driver' not in colunas:
        df['Age_band_of_driver'] = df['Age_band_of_driver'].replace({"Unknown" : 0,
                                                                             "Under 18": 1,
                                                                             "18-30": 2,
                                                                             "31-50": 3,
                                                                             "Over 51": 4})
    if 'Sex_of_driver' not in colunas:
        df['Sex_of_driver'] = df['Sex_of_driver'].replace({"Unknown": 0,
                                                                   "Female": 1,
                                                                   "Male": 2})
    if 'Educational_level' not in colunas:
        df['Educational_level'] = df['Educational_level'].replace({"Unknown": 0,
                                                                           "Illiterate": 1,
                                                                           "Writing & reading": 2,
                                                                           "Elementary school": 3,
                                                                           "Junior high school": 4,
                                                                           "High school": 5,
                                                                           "Above high school": 6})
    if 'Vehicle_driver_relation' not in colunas:
        df['Vehicle_driver_relation'] = df['Vehicle_driver_relation'].replace({"Unknown": 0,
                                                                                       "Other": 0,
                                                                                       "Employee": 1,
                                                                                       "Owner": 2})
    
    if 'Driving_experience' not in colunas:

        df['Driving_experience'] = df['Driving_experience'].replace({"unknown": 0,
                                                                             "No Licence": 1,
                                                                             "Below 1yr": 2,
                                                                             "1-2yr": 3,
                                                                             "2-5yr": 4,
                                                                             "5-10yr": 5,
                                                                             "Above 10yr": 6})
    if 'Type_of_vehicle' not in colunas:
        df['Type_of_vehicle'] = df['Type_of_vehicle'].replace({"Other": 0,
                                                                       "Public (> 45 seats)": 1,
                                                                       "Public (13?45 seats)": 1,
                                                                       "Public (12 seats)": 1,
                                                                       "Lorry (41?100Q)" : 2,
                                                                       "Lorry (11?40Q)": 2,
                                                                       "Long lorry": 2,
                                                                       "Automobile": 3,
                                                                       "Taxi": 4,
                                                                       "Pick up upto 10Q": 5,
                                                                       "Stationwagen": 6,
                                                                       "Ridden horse": 7,
                                                                       "Bajaj": 8,
                                                                       "Motorcycle": 8,
                                                                       "Turbo":9,
                                                                       "Special vehicle" : 10,
                                                                       "Bicycle": 11})
    if 'Owner_of_vehicle' not in colunas:
        df['Owner_of_vehicle'] = df['Owner_of_vehicle'].replace({"Other": 0,
                                                                         "Owner": 1,
                                                                         "Organization": 2,
                                                                         "Governmental": 3})
    
    if 'Area_accident_occured' not in colunas:
        df['Area_accident_occured'] = df['Area_accident_occured'].replace({"Unknown": 0,
                                                                                   "Other": 0,
                                                                                   "Rural village areasOffice areas": 0,
                                                                                   "  Recreational areas": 1,
                                                                                   "Recreational areas": 1,
                                                                                   "Residential areas": 2,
                                                                                   "Office areas": 3,
                                                                                   " Industrial areas": 4,
                                                                                   " Church areas": 5,
                                                                                   "  Market areas": 6,
                                                                                   "Rural village areas": 7,
                                                                                   " Outside rural areas": 8,
                                                                                   " Hospital areas": 9,
                                                                                  "School areas": 10})
    if 'Service_year_of_vehicle' not in colunas:    
        df['Service_year_of_vehicle'] = df['Service_year_of_vehicle'].replace({"Unknown" : 0,
                                                                           "Below 1yr" : 1,
                                                                           "1-2yr" : 2,
                                                                           "2-5yrs" : 3,
                                                                           "5-10yrs" : 4,
                                                                           "Above 10yr" : 5})
    if 'Defect_of_vehicle' not in colunas:    
        df['Defect_of_vehicle'] = df['Defect_of_vehicle'].replace({"No defect" : 0,
                                                               "5" : 1,
                                                               "7" : 2})
        
    if 'Lanes_or_Medians' not in colunas: 
        df['Lanes_or_Medians'] = df['Lanes_or_Medians'].replace({"Unknown" : 0,
                                                                         "other": 0,
                                                                         "Undivided Two way": 1,
                                                                         "Double carriageway (median)": 2,
                                                                         "Two-way (divided with solid lines road marking)": 3,
                                                                         "Two-way (divided with broken lines road marking)": 4,
                                                                         "One way": 5})
    if 'Road_allignment' not in colunas: 
        df['Road_allignment'] = df['Road_allignment'].replace({"Tangent road with flat terrain": 0,
                                                                       "Tangent road with mild grade and flat terrain": 1,
                                                                       "Tangent road with rolling terrain": 2,
                                                                       "Tangent road with mountainous terrain and": 3,
                                                                       "Escarpments": 4,
                                                                       "Gentle horizontal curve": 5,
                                                                       "Steep grade downward with mountainous terrain": 6,
                                                                       "Sharp reverse curve": 7,
                                                                       "Steep grade upward with mountainous terrain": 8})
    
    if 'Types_of_Junction' not in colunas: 
        df['Types_of_Junction'] = df['Types_of_Junction'].replace({"Unknown": 0,
                                                                           "Other": 0,
                                                                           "No junction": 1,
                                                                           "Y Shape": 2,
                                                                           "Crossing": 3,
                                                                           "O Shape": 4,
                                                                           "T Shape": 5,
                                                                           "X Shape": 6})
    if 'Road_surface_type' not in colunas:
        df['Road_surface_type'] = df['Road_surface_type'].replace({"Other": 0,
                                                                           "Asphalt roads": 1, 
                                                                           "Asphalt roads with some distress": 2,
                                                                           "Gravel roads": 3,
                                                                           "Earth roads": 4})
    
    if 'Road_surface_conditions' not in colunas:
        df['Road_surface_conditions'] = df['Road_surface_conditions'].replace({"Dry": 0,
                                                                                       "Wet or damp": 1,
                                                                                       "Snow": 2,
                                                                                       "Flood over 3cm. deep": 3})
    
    if 'Light_conditions' not in colunas:
        df['Light_conditions'] = df['Light_conditions'].replace({"Daylight": 0,
                                                                         "Darkness - lights lit": 1,
                                                                         "Darkness - no lighting": 2,
                                                                         "Darkness - lights unlit": 3})
    if 'Weather_conditions' not in colunas:
        df['Weather_conditions'] = df['Weather_conditions'].replace({"Unknown": 0,
                                                                             "Other": 0,
                                                                             "Normal": 1,
                                                                             "Raining": 2,
                                                                             "Raining and Windy": 3,
                                                                             "Cloudy": 4,
                                                                             "Windy": 5,
                                                                             "Snow": 6,
                                                                             "Fog or mist": 7})
    if 'Type_of_collision' not in colunas:
        df['Type_of_collision'] = df['Type_of_collision'].replace({"Unknown": 0,
                                                                           "Other": 0,
                                                                           "Collision with roadside-parked vehicles":1,
                                                                           "Vehicle with vehicle collision":2,
                                                                           "Collision with roadside objects":3,
                                                                           "Collision with animals": 4,
                                                                           "Rollover": 5,
                                                                           "Fall from vehicles": 6,
                                                                           "Collision with pedestrians": 7,
                                                                           "With Train": 8})
    if 'Vehicle_movement' not in colunas:
        df['Vehicle_movement'] = df['Vehicle_movement'].replace({"Unknown": 0,
                                                                         "Other": 0,
                                                                         "Going straight": 1,
                                                                         "U-Turn": 2,
                                                                         "Moving Backward": 3,
                                                                         "Turnover": 4,
                                                                         "Waiting to go": 5,
                                                                         "Getting off": 6,
                                                                         "Reversing": 7,
                                                                         "Parked": 8,
                                                                         "Stopping": 9,
                                                                         "Overtaking": 10,
                                                                         "Entering a junction": 11})
    
    if 'Casualty_class' not in colunas:
        df['Casualty_class'] = df['Casualty_class'].replace({"na": 0,
                                                                     "Driver or rider": 1,
                                                                     "Pedestrian": 2,
                                                                     "Passenger": 3})
    
    if 'Sex_of_casualty' not in colunas:
        df['Sex_of_casualty'] = df['Sex_of_casualty'].replace({"na": 0,
                                                                       "Female": 1,
                                                                       "Male":2})
    if 'Age_band_of_casualty' not in colunas:
        df['Age_band_of_casualty'] = df['Age_band_of_casualty'].replace({"na":0,
                                                                                 "5": 1,
                                                                                 "Under 18": 1,
                                                                                 "18-30": 2,
                                                                                 "31-50": 3,
                                                                                 "Over 51": 4})
    
    if 'Casualty_severity' not in colunas:
        df['Casualty_severity'] = df['Casualty_severity'].replace({"na": 0,
                                                                           "1": 1,
                                                                           "2": 2,
                                                                           "3": 3})
    if 'Work_of_casuality' not in colunas:
        df['Work_of_casuality'] = df['Work_of_casuality'].replace({"Unknown" : 0,
                                                               "Other": 0,
                                                               "Student": 1,
                                                               "Unemployed": 2,
                                                               "Employee": 3,
                                                               "Driver": 4,
                                                               "Self-employed": 5})
        
    if 'Fitness_of_casuality' not in colunas:
        df['Fitness_of_casuality'] = df['Fitness_of_casuality'].replace({"Other" : 0,
                                                                     "Normal" : 1,
                                                                     "NormalNormal": 1,
                                                                     "Deaf" : 3,
                                                                     "Blind" : 4})
    
    if 'Pedestrian_movement' not in colunas:
        df['Pedestrian_movement'] = df['Pedestrian_movement'].replace({"Unknown or other":0,
                                                                    "Not a Pedestrian": 1,
                                                                    "Crossing from driver's nearside": 2,
                                                                    "Crossing from nearside - masked by parked or statioNot a Pedestrianry vehicle": 3,
                                                                    "Crossing from offside - masked by  parked or statioNot a Pedestrianry vehicle": 4,
                                                                    "In carriageway, statioNot a Pedestrianry - not crossing  (standing or playing)": 5,
                                                                    "Walking along in carriageway, back to traffic": 6,
                                                                    "Walking along in carriageway, facing traffic": 7,
                                                                    "In carriageway, statioNot a Pedestrianry - not crossing  (standing or playing) - masked by parked or statioNot a Pedestrianry vehicle": 8})
    
    if 'Cause_of_accident' not in colunas:
        df['Cause_of_accident'] = df['Cause_of_accident'].replace({"Unknown": 0,
                                                                           "Other": 0,
                                                                           "Moving Backward": 1,
                                                                           "Overtaking": 2,
                                                                           "Changing lane to the left": 3,
                                                                           "Changing lane to the right": 4,
                                                                           "Overloading": 5,
                                                                           "No priority to vehicle": 6,
                                                                           "No priority to pedestrian": 7,
                                                                           "No distancing": 8,
                                                                           "Getting off the vehicle improperly": 9,
                                                                           "Improper parking": 10,
                                                                           "Overspeed": 11,
                                                                           "Driving carelessly": 12,
                                                                           "Driving at high speed": 13,
                                                                           "Driving to the left": 14,
                                                                           "Overturning": 15,
                                                                           "Turnover": 16,
                                                                           "Driving under the influence of drugs": 17,
                                                                           "Drunk driving": 18})
    if 'Accident_severity' not in colunas:
        df['Accident_severity'] = df['Accident_severity'].replace({"Slight Injury": 0,
                                                                           "Serious Injury": 1,
                                                                           "Fatal injury": 2})


    print(df)
    return df




In [39]:
from sklearn.preprocessing import OneHotEncoder

def onehot(df, colunas_tiradas): 

    enc = OneHotEncoder(sparse_output=False)
    categorical_columns = ['Sex_of_driver', 'Vehicle_driver_relation', 'Type_of_vehicle', 'Owner_of_vehicle', 'Defect_of_vehicle','Area_accident_occured','Lanes_or_Medians','Road_allignment', 'Types_of_Junction','Owner_of_vehicle', 'Road_surface_type','Light_conditions', 'Weather_conditions', 'Type_of_collision','Vehicle_movement','Casualty_class','Sex_of_casualty', 'Work_of_casuality', 'Fitness_of_casuality', 'Pedestrian_movement', 'Cause_of_accident']
    categorical_columns = [col for col in categorical_columns if col not in colunas_tiradas]
    one_hot_encoded = enc.fit_transform(df[categorical_columns])
    categorical_columns_array = (['Sex_of_driver', 'Vehicle_driver_relation', 'Type_of_vehicle', 'Owner_of_vehicle', 'Defect_of_vehicle','Area_accident_occured','Lanes_or_Medians','Road_allignment', 'Types_of_Junction','Owner_of_vehicle', 'Road_surface_type','Light_conditions', 'Weather_conditions', 'Type_of_collision','Vehicle_movement','Casualty_class','Sex_of_casualty', 'Work_of_casuality', 'Fitness_of_casuality', 'Pedestrian_movement', 'Cause_of_accident'])
    categorical_columns_array = categorical_columns_array = [col for col in categorical_columns_array if col not in colunas_tiradas]
    one_hot_df = pd.DataFrame(one_hot_encoded, columns=enc.get_feature_names_out(categorical_columns_array))
    df_encoded = pd.concat([df, one_hot_df], axis=1)
    df_encoded = df_encoded.drop(categorical_columns, axis=1)
    print(df_encoded)
    print(categorical_columns)
    return df_encoded




In [ ]:
df_semmv, colunas_tiradas = tratar_missing_values(df)
df_numerico = tornarnuméricoA(df_semmv, colunas_tiradas)
df_processado = onehot(df_numerico, colunas_tiradas)


In [41]:
outliers = {}
def detetaroutliers():
    for coluna in df_processado:
        Q1 = df_sem_outliers[coluna].quantile(0.25)  
        Q3 = df_sem_outliers[coluna].quantile(0.75)  
        IQR = Q3 - Q1 
        
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
    
        outlier_indices = df_sem_outliers[(df_sem_outliers[coluna] < limite_inferior) | (df_sem_outliers[coluna] > limite_superior)].index
        outliers[coluna] = outlier_indices

        df_sem_outliers = df_sem_outliers[(df_sem_outliers[coluna] >= limite_inferior) & (df_sem_outliers[coluna] <= limite_superior)]
        
        num_outliers = len(outliers)  
        total_valores = len(df_processado[coluna]) 
        print((num_outliers/total_valores)* 100) # % de outliers
        
        print(f"Coluna '{coluna}':")
        print(f"  Limite Inferior: {limite_inferior}, Limite Superior: {limite_superior}")
        print(f"  Número de outliers: {len(outlier_indices)}\n")

## Experimental Analysis

### Scenario A (Binary Classification)

In [42]:
X = df_numerico.drop('Accident_severity', axis=1)
y = df_numerico['Accident_severity']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X_train.columns


In [ ]:
# entender que features são mais relevantes através do cálculo da variância explicada
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.preprocessing import StandardScaler


column_names = ['Time', 'Day_of_week', 'Age_band_of_driver', 'Sex_of_driver',
       'Educational_level', 'Vehicle_driver_relation', 'Driving_experience',
       'Type_of_vehicle', 'Owner_of_vehicle', 'Area_accident_occured',
       'Lanes_or_Medians', 'Road_allignment', 'Types_of_Junction',
       'Road_surface_type', 'Road_surface_conditions', 'Light_conditions',
       'Weather_conditions', 'Type_of_collision',
       'Number_of_vehicles_involved', 'Number_of_casualties',
       'Vehicle_movement', 'Casualty_class', 'Sex_of_casualty',
       'Age_band_of_casualty', 'Casualty_severity', 'Pedestrian_movement',
       'Cause_of_accident']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test)  


pca = PCA(n_components=0.90)  


X_pca = pca.fit_transform(X_train_scaled) 
n_components = pca.n_components_


print(f'Número de componentes: {n_components}')
print(f'Variância explicada por componente: {pca.explained_variance_ratio_}')
print(f'Variância total explicada: {sum(pca.explained_variance_ratio_)}')


components_df = pd.DataFrame(pca.components_, columns=column_names) 


print("\nFeatures mais relevantes para cada componente principal:")
for i in range(n_components):
    print(f"\nComponente {i+1}:")
    print(components_df.iloc[i].abs().sort_values(ascending=False).head(10))




  

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)


In [21]:
from imblearn.over_sampling import SMOTE

def aumentardados(X_train, y_train):
    smote = SMOTE(random_state=90, sampling_strategy=0.5) # a classe minoritária tem metade dos dados da classe maioritária
    X_res, y_res = smote.fit_resample(X_train, y_train)
    return X_res, y_res

In [22]:
def metricasaval(y_test, y_test_pred):
    from sklearn import metrics
    from sklearn.metrics import accuracy_score, classification_report
    
    print('Misclassified samples: %d' % (y_test != y_test_pred).sum())
    misclassified_indices = np.where(y_test != y_test_pred)[0]
    print('Accuracy: %.2f' % accuracy_score(y_test, y_test_pred))
    print(classification_report(y_test, y_test_pred))
    cm_test = metrics.confusion_matrix(y_test, y_test_pred)
    print(cm_test)
    sns.heatmap(cm_test, annot=True, fmt="d", cmap="Blues", xticklabels=mlp.classes_, yticklabels=mlp.classes_)
    plt.title("Confusion Matrix")
    plt.show()
    return misclassified_indices
    

MLP com GridSearch 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.neural_network import MLPClassifier

def nngrid(X_train, X_val, X_test, y_train, y_val, y_test):

   

    parameters = {
        'hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 100)],
        'activation': ['relu', 'tanh'],
        'batch_size': [8, 16],
        'learning_rate_init': [0.0001, 0.001],
    }

    mlp = MLPClassifier(max_iter=10000, tol=0.01, random_state=1)
    clf = GridSearchCV(mlp, param_grid=parameters, scoring='f1', cv=3, verbose=3)
    clf.fit(X_train, y_train)
    y_val_pred = clf.predict(X_val)
    val_accuracy = metrics.accuracy_score(y_val, y_val_pred)
    val_report = metrics.classification_report(y_val, y_val_pred)
    y_test_pred = clf.predict(X_test)
    test_cm = metrics.confusion_matrix(y_test, y_test_pred)

    print('Best parameters:', clf.best_params_)
    print('Class labels:', np.unique(y_test))
    print('Misclassified samples: %d' % (y_test != y_test_pred).sum())
    print('Test Accuracy: %.2f' % metrics.accuracy_score(y_test, y_test_pred))
    print(metrics.classification_report(y_test, y_test_pred))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=clf.classes_, yticklabels=clf.classes_)
    plt.show()

    
nngrid(X_train, X_test, y_train, y_test)

MLP com parametros ajustados manualmente - MLP5

In [24]:
#MLP5 - modelo com melhor performance conseguida com este algoritmo
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100, 100), learning_rate_init=0.001, max_iter=10000, random_state=3, batch_size=8, n_iter_no_change = 20)

In [ ]:
X_res, y_res = aumentardados(X_train, y_train)

mlp.fit(X_res, y_res)

y_test_proba = mlp.predict_proba(X_test)

y_test_pred = (y_test_proba[:, 1] > 0.2).astype(int)


metricasaval(y_test, y_test_pred)


In [ ]:
X_res, y_res = aumentardados(X_train, y_train)
mlp.fit(X_res, y_res)
y_test_pred = mlp.predict(X_test)
metricasaval(y_test, y_test_pred)


### SVM

RandomSearch (não terminou)

In [ ]:
"""
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Define os hiperparâmetros para busca
param_dist = {
    'C': np.logspace(-3, 3, 10),           # Regularização
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Tipos de kernel
    'gamma': ['scale', 'auto'],            # Coeficiente do kernel
    'degree': [2, 3, 4],                   # Para kernel 'poly'
    'coef0': [0.0, 0.1, 0.5]               # Para 'poly' e 'sigmoid'
}

# Define o modelo
svc = SVC()

# Configura o RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=svc,
    param_distributions=param_dist,
    n_iter=20,              # Número de combinações de parâmetros a testar
    scoring='f1_weighted',     # Métrica de avaliação
    cv=3,                   # Número de folds para validação cruzada
    random_state=42,        # Para resultados reprodutíveis
    verbose=3,              # Para acompanhar o progresso
    n_jobs=-1               # Usa todos os núcleos disponíveis
)

# Executa o ajuste

random_search.fit(X_train, y_train)

# Resultados
print("Melhores parâmetros:", random_search.best_params_)
print("Melhor precisão na validação cruzada: %.2f" % random_search.best_score_)

# Avaliação no conjunto de teste
best_model = random_search.best_estimator_
y_val_pred = best_model.predict(X_test)
y_pred = best_model.predict(X_test)

metricasaval(y_test, y_test_pred)
"""


SVM8 - modelo selecionado como melhor performance com este algoritmo

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np


clf = SVC(random_state=2, kernel='rbf', gamma=0.0003, C=15, class_weight = {0: 1, 1: 3})
clf.fit(X_res, y_res)


y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
misclassified_indices = np.where(y_test != y_pred)[0]  


print('Misclassified samples: %d' % len(misclassified_indices))
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", cm)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=clf.classes_, yticklabels=clf.classes_)
plt.title("Confusion Matrix")
plt.show()

### Ensemble

Ensemble6 - modelo selecionado como melhor performance com este algoritmo

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

X_res, y_res = aumentardados(X_train, y_train)

svm1 = SVC(random_state = 2, kernel='rbf', C=15, probability=True)
svm2 = SVC(random_state=2, kernel='rbf', C=0.01, class_weight = {0: 1, 1: 50}, probability=True)
svm3 = SVC(random_state=2, kernel='rbf', gamma=0.0003, C=15, class_weight = {0: 1, 1: 3}, probability=True)
svm4 = SVC(random_state=2, kernel='linear', probability=True)



ensemble = VotingClassifier(
    estimators=[('svm1', svm1), ('svm2', svm2), ('svm3', svm3), ('svm4', svm4) ],
    voting='soft',  
    weights=[2, 1, 2, 1]  
)

# Treine o ensemble
ensemble.fit(X_res, y_res)

y_pred_proba = ensemble.predict_proba(X_test)[:, 1]  

limiar = 0.3 
y_pred_adjusted = (y_pred_proba > limiar).astype(int) 


print(classification_report(y_test, y_pred_adjusted))
cm = confusion_matrix(y_test, y_pred_adjusted)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

## Scenario B (Multiclasse)

In [ ]:
df_semmv, colunas_tiradas = tratar_missing_values(df)
df_numerico2 = tornarnuméricoB(df_semmv, colunas_tiradas)
df_processado2 = onehot(df_numerico2, colunas_tiradas)
print(df_processado2)
print(df_processado2['Accident_severity'].unique())

In [ ]:
print(df_processado2['Accident_severity'].unique())

In [ ]:
print(len(df_processado2[df_processado2['Accident_severity'] == 2]))
len(df_processado2[df_processado2['Accident_severity'] == 1])


In [170]:
X = df_processado2.drop('Accident_severity', axis=1)
y = df_processado2['Accident_severity']

In [172]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [189]:
from imblearn.over_sampling import SMOTE

def aumentardados2(X_train, y_train):
    smote = SMOTE(random_state=90, sampling_strategy={1: 2200, 2: 400}) # a classe minoritária tem metade dos dados da classe maioritária
    X_res, y_res = smote.fit_resample(X_train, y_train)
    return X_res, y_res

MLP5

In [190]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation ='relu', batch_size = 8, hidden_layer_sizes = (100,), learning_rate_init = 0.001 )

In [ ]:
from sklearn.neural_network import MLPClassifier


X_res, y_res = aumentardados2(X_train, y_train)


mlp.fit(X_res, y_res)

y_test_proba = mlp.predict_proba(X_test)


print("Probabilidades para a classe 1:", y_test_proba[:, 2])
y_test_pred = (y_test_proba[:, 1] > 0.2).astype(int)

metricasaval(y_test, y_test_pred)

SVM8

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np


clf = SVC(random_state=2, kernel='rbf', gamma=0.0003, C=15, class_weight = {0: 1, 1: 3})
clf.fit(X_res, y_res)


y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
misclassified_indices = np.where(y_test != y_pred)[0]  


print('Misclassified samples: %d' % len(misclassified_indices))
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", cm)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=clf.classes_, yticklabels=clf.classes_)
plt.title("Confusion Matrix")
plt.show()

Ensemble3

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

svm1 = SVC(random_state = 2, kernel='rbf', C=15, probability=True)
svm2 = SVC(random_state=2, kernel='rbf', C=0.01, class_weight = {0: 1, 1: 50}, probability=True)
svm3 = SVC(random_state=2, kernel='rbf', gamma=0.0003, C=15, class_weight = {0: 1, 1: 3}, probability=True)
svm4 = SVC(random_state=2, kernel='linear', probability=True)



ensemble = VotingClassifier(
    estimators=[('svm1', svm1), ('svm2', svm2), ('svm3', svm3), ('svm4', svm4) ],
    voting='soft',  
    weights=[2, 1, 2, 1]  
)

# Treine o ensemble
ensemble.fit(X_res, y_res)

y_pred_proba = ensemble.predict_proba(X_test)[:, 1]  

limiar = 0.3 
y_pred_adjusted = (y_pred_proba > limiar).astype(int) 


print(classification_report(y_test, y_pred_adjusted))
cm = confusion_matrix(y_test, y_pred_adjusted)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

Ensemble RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

rf1 = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf2 = RandomForestClassifier(n_estimators=150, random_state=42, max_depth=10, class_weight='balanced')
rf3 = RandomForestClassifier(n_estimators=200, random_state=42, max_features='sqrt', class_weight='balanced')

ensemble_rf = VotingClassifier(
    estimators=[('rf1', rf1), ('rf2', rf2), ('rf3', rf3)],
    voting='soft'  
)


ensemble_rf.fit(X_res, y_res)

y_pred = ensemble_rf.predict(X_test)


print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix - Ensemble Random Forest")
plt.show()
